<a href="https://colab.research.google.com/github/Pavel184/Neural_networks/blob/Homework_lesson_3/Homework_lesson_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework_lesson_3

1. Постройте нейронную сеть(берем простую линейную сеть, которую разбирали на уроке: меняем число слоев, число нейронов , типы активации, тип оптимизатора)  на датасет from sklearn.datasets import load_boston. 

In [ ]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
#from tensorflow.keras.datasets import mnist
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
#from tensorflow.keras.utils import to_categorical
import plotly.graph_objects as go

In [ ]:
# Загрузка датасета и разделение на train test
X, y = load_boston(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, 
                                                    random_state=42)

In [ ]:
# размер входных данных
dim = X_train.shape[1]

In [ ]:
# Масштабирование данных
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

In [ ]:
# Построение модели
model = Sequential()
model.add(Dense(128, input_shape=(dim, ), activation='relu', name='dense_1'))
model.add(Dense(64, activation='relu', name='dense_2'))
model.add(Dense(1, activation='linear', name='dense_output'))
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               1792      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_output (Dense)         (None, 1)                 65        
Total params: 10,113
Trainable params: 10,113
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Обучение модели
hist = model.fit(X_train, y_train, epochs=100, validation_split=0.05, verbose=0)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scattergl(y=hist.history['loss'],
                    name='Train'))
fig.add_trace(go.Scattergl(y=hist.history['val_loss'],
                    name='Valid'))
fig.update_layout(height=500, width=700,
                  xaxis_title='Epoch',
                  yaxis_title='Loss')
fig.show()

In [ ]:
# Метрики
mse_nn, mae_nn = model.evaluate(X_test, y_test)
print('MSE на тестовых данных: ', mse_nn)
print('MAE на тестовых данных: ', mae_nn)

6/6 [==============================] - 0s 3ms/step - loss: 10.7036 - mae: 2.1747
MSE на тестовых данных:  10.703579902648926
MAE на тестовых данных:  2.174665689468384


2. Измените функцию потерь и метрику для этой задачи. Постройте 10-15 вариантов и сведите результаты их работы в таблицу  Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?

In [ ]:
# Таблица результатов
models_results = {
    'approach': [],
    'activation_func': [],
    'metrics': [],
    'neurons_number': [],
    'epochs_numb': [],
    'type_of_loss': [],
    'optimizer': [],
    'MSE_test': [],
    'MAE_test': []
}

In [ ]:
# Модели с разными параметрами
epochs_numb = [100, 200, 500]
loss_list = ['mean_squared_error', 'mean_absolute_error', 
        'mean_absolute_percentage_error', 'mean_squared_logarithmic_error',
        'cosine_similarity']
optimizer_list = ['adam', 'Nadam', 'SGD', 'RMSprop', 'Ftrl', 'Adamax', 
                  'Adagrad', 'Adadelta']
neurons_numb = [[64,32], [128,64], [256,128], [512, 256]]
metrics_list = ['mae', 'mse']
act_fn_list = ['relu', 'sigmoid', 'tanh', 'elu']
for epochs in epochs_numb:
  for loss in loss_list:
    for optimizer in optimizer_list:
      for n_numb in neurons_numb:
        for metrics in metrics_list:
          for activation in act_fn_list:
            model_1 = Sequential()
            model_1.add(Dense(n_numb[0], input_shape=(dim, ), activation='relu', name='dense_1'))
            model_1.add(Dense(n_numb[1], activation=activation, name='dense_2'))
            model_1.add(Dense(1, activation='linear', name='dense_output'))
            model_1.compile(optimizer=optimizer, loss=loss, metrics=metrics)

            model_1.fit(X_train, y_train, epochs=epochs, validation_split=0.05, verbose=0)

            mse_nn, mae_nn = model_1.evaluate(X_test, y_test, verbose=0)

            models_results['approach'].append('2 layers model')
            models_results['activation_func'].append(activation)
            models_results['metrics'].append(metrics)
            models_results['neurons_number'].append(n_numb)
            models_results['epochs_numb'].append(epochs)
            models_results['type_of_loss'].append(loss)
            models_results['optimizer'].append(optimizer)
            models_results['MSE_test'].append(mse_nn)
            models_results['MAE_test'].append(mae_nn)

In [ ]:
# Итоговая таблица
df_1 = pd.DataFrame(data=models_results).sort_values('MSE_test', ascending=True)
df_1.head(100)

,approach,activation_func,metrics,neurons_number,epochs_numb,type_of_loss,optimizer,MSE_test,MAE_test
2455,2 layers model,elu,mse,"[256, 128]",200,cosine_similarity,Ftrl,-1.0,544.085510
1229,2 layers model,sigmoid,mse,"[128, 64]",100,cosine_similarity,Adagrad,-1.0,511.950226
1237,2 layers model,sigmoid,mse,"[256, 128]",100,cosine_similarity,Adagrad,-1.0,525.993164
1241,2 layers model,sigmoid,mae,"[512, 256]",100,cosine_similarity,Adagrad,-1.0,20.983261
1265,2 layers model,sigmoid,mae,"[256, 128]",100,cosine_similarity,Adadelta,-1.0,21.536509
...,...,...,...,...,...,...,...,...,...
2477,2 layers model,sigmoid,mse,"[128, 64]",200,cosine_similarity,Adamax,-1.0,485.905975
2476,2 layers model,relu,mse,"[128, 64]",200,cosine_similarity,Adamax,-1.0,383.149445
2475,2 layers model,elu,mae,"[128, 64]",200,cosine_similarity,Adamax,-1.0,16.566603
2474,2 layers model,tanh,mae,"[128, 64]",200,cosine_similarity,Adamax,-1.0,17.194485


In [ ]:
# Убираем отрецательные значения
df_2 =df_1.drop(df_1[df_1.MSE_test < 0].index)
df_2.head(30)

,approach,activation_func,metrics,neurons_number,epochs_numb,type_of_loss,optimizer,MSE_test,MAE_test
1103,2 layers model,elu,mse,"[128, 64]",100,cosine_similarity,SGD,0.005988,540.036926
3350,2 layers model,tanh,mse,"[256, 128]",500,mean_squared_logarithmic_error,adam,0.017484,9.499741
2514,2 layers model,tanh,mae,"[256, 128]",200,cosine_similarity,Adagrad,0.017964,21.621441
2370,2 layers model,tanh,mae,"[64, 32]",200,cosine_similarity,SGD,0.017964,21.694885
3433,2 layers model,sigmoid,mae,"[128, 64]",500,mean_squared_logarithmic_error,RMSprop,0.018873,2.121380
3378,2 layers model,tanh,mae,"[256, 128]",500,mean_squared_logarithmic_error,Nadam,0.019159,2.161700
2153,2 layers model,sigmoid,mae,"[128, 64]",200,mean_squared_logarithmic_error,RMSprop,0.020121,2.159405
3424,2 layers model,relu,mae,"[64, 32]",500,mean_squared_logarithmic_error,RMSprop,0.020266,2.083420
3332,2 layers model,relu,mse,"[64, 32]",500,mean_squared_logarithmic_error,adam,0.020427,10.281829
3346,2 layers model,tanh,mae,"[256, 128]",500,mean_squared_logarithmic_error,adam,0.020571,2.165648


Лучший результат MSE 0.018873 MAE 2.121380

In [ ]:
# Построим модель наилучшего результата
model = Sequential()
model.add(Dense(128, input_shape=(dim, ), activation='sigmoid', name='dense_1'))
model.add(Dense(64, activation='sigmoid', name='dense_2'))
model.add(Dense(1, activation='linear', name='dense_output'))
model.compile(optimizer='RMSprop', loss='mean_squared_logarithmic_error', metrics=['mae'])
model.summary()
# Обучение модели
hist = model.fit(X_train, y_train, epochs=500, validation_split=0.05, verbose=0)
fig = go.Figure()
fig.add_trace(go.Scattergl(y=hist.history['loss'],
                    name='Train'))
fig.add_trace(go.Scattergl(y=hist.history['val_loss'],
                    name='Valid'))
fig.update_layout(height=500, width=700,
                  xaxis_title='Epoch',
                  yaxis_title='Loss')
fig.show()
# Метрики
mse_nn, mae_nn = model.evaluate(X_test, y_test)
print('MSE на тестовых данных: ', mse_nn)
print('MAE на тестовых данных: ', mae_nn)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               1792      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_output (Dense)         (None, 1)                 65        
Total params: 10,113
Trainable params: 10,113
Non-trainable params: 0
_________________________________________________________________


6/6 [==============================] - 0s 3ms/step - loss: 0.0285 - mae: 2.3189
MSE на тестовых данных:  0.02846105583012104
MAE на тестовых данных:  2.3188869953155518


In [ ]:
y_test

array([23.6, 32.4, 13.6, 22.8, 16.1, 20. , 17.8, 14. , 19.6, 16.8, 21.5,
       18.9,  7. , 21.2, 18.5, 29.8, 18.8, 10.2, 50. , 14.1, 25.2, 29.1,
       12.7, 22.4, 14.2, 13.8, 20.3, 14.9, 21.7, 18.3, 23.1, 23.8, 15. ,
       20.8, 19.1, 19.4, 34.7, 19.5, 24.4, 23.4, 19.7, 28.2, 50. , 17.4,
       22.6, 15.1, 13.1, 24.2, 19.9, 24. , 18.9, 35.4, 15.2, 26.5, 43.5,
       21.2, 18.4, 28.5, 23.9, 18.5, 25. , 35.4, 31.5, 20.2, 24.1, 20. ,
       13.1, 24.8, 30.8, 12.7, 20. , 23.7, 10.8, 20.6, 20.8,  5. , 20.1,
       48.5, 10.9,  7. , 20.9, 17.2, 20.9,  9.7, 19.4, 29. , 16.4, 25. ,
       25. , 17.1, 23.2, 10.4, 19.6, 17.2, 27.5, 23. , 50. , 17.9,  9.6,
       17.2, 22.5, 21.4, 12. , 19.9, 19.4, 13.4, 18.2, 24.6, 21.1, 24.7,
        8.7, 27.5, 20.7, 36.2, 31.6, 11.7, 39.8, 13.9, 21.8, 23.7, 17.6,
       24.4,  8.8, 19.2, 25.3, 20.4, 23.1, 37.9, 15.6, 45.4, 15.7, 22.6,
       14.5, 18.7, 17.8, 16.1, 20.6, 31.6, 29.1, 15.6, 17.5, 22.5, 19.4,
       19.3,  8.5, 20.6, 17. , 17.1, 14.5, 50. , 14

In [ ]:
y_pred = model.predict(X_test)
y_pred

array([[23.41915  ],
       [31.720438 ],
       [13.114435 ],
       [23.124352 ],
       [14.055444 ],
       [20.181738 ],
       [17.633347 ],
       [14.07095  ],
       [22.689331 ],
       [18.888199 ],
       [19.722569 ],
       [18.018784 ],
       [ 8.275054 ],
       [20.068077 ],
       [18.657694 ],
       [23.049606 ],
       [19.246357 ],
       [ 9.88753  ],
       [43.013813 ],
       [11.99634  ],
       [23.74474  ],
       [25.25003  ],
       [14.42342  ],
       [21.563934 ],
       [16.526196 ],
       [15.659239 ],
       [20.68753  ],
       [10.775575 ],
       [18.946312 ],
       [18.446716 ],
       [19.442429 ],
       [22.852234 ],
       [19.458496 ],
       [23.572289 ],
       [12.491953 ],
       [15.637371 ],
       [29.755348 ],
       [19.94889  ],
       [20.45739  ],
       [23.610598 ],
       [16.543102 ],
       [28.565825 ],
       [44.74477  ],
       [18.642258 ],
       [22.530983 ],
       [14.98048  ],
       [15.642636 ],
       [25.04

3. Поработайте с документацией TensorFlow 2. Найти 2-3 полезные команды TensorFlow, не разобранные на уроке (полезные для Вас).


1. tf.estimator.BaselineClassifier, tf.estimator.BaselineRegressor для быстрого построения базовых

2. tf.keras.utils.text_dataset_from_directory собирает датасет из текстовых файлов